# Retrieval Agumented Generation

In [1]:
import faiss
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer

embed_model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

You try to use a model that was created with version 2.4.0.dev0, however, your version is 2.3.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



<All keys matched successfully>


In [3]:
# language_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
sentences = ['search_query: What is TSNE?', 'search_query: Who is Laurens van der Maaten?']
embeddings = embed_model.encode(sentences)
print(embeddings)

[[ 1.0951382e-02  5.7414606e-02 -1.1036388e-02 ...  3.5131579e-05
  -2.8092245e-02 -2.1599913e-02]
 [-1.3367120e-02  2.7091343e-02 -2.3367403e-02 ...  2.8799422e-02
  -1.0674847e-02  2.8820729e-02]]
